# 使用 Mistral 模型建構應用

## 介紹

本課程將會涵蓋：
- 探索不同的 Mistral 模型
- 了解每個模型的使用情境與適用場合
- 程式碼範例展示各模型的獨特功能


## Mistral 模型

在這一課，我們會介紹三款不同的 Mistral 模型：**Mistral Large**、**Mistral Small** 和 **Mistral Nemo**。

這些模型都可以在 Github Model marketplace 免費取得。這份筆記本裡的程式碼會用這些模型來執行。你可以參考這裡，了解更多如何用 Github Models 來[用 AI 模型做原型開發](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)。


## Mistral Large 2 (2407)
Mistral Large 2 目前是 Mistral 的旗艦模型，專為企業級應用設計。

這個模型是原始 Mistral Large 的升級版，提供了
- 更大的上下文視窗——128k 對比 32k
- 更優異的數學與程式碼任務表現——平均準確率 76.9% 對比 60.4%
- 多語言能力提升——支援語言包括：英文、法文、德文、西班牙文、義大利文、葡萄牙文、荷蘭文、俄文、中文、日文、韓文、阿拉伯文和印地語。

有了這些特點，Mistral Large 在以下方面表現出色：
- *檢索增強生成（RAG）*——因為有更大的上下文視窗
- *函式呼叫*——這個模型原生支援函式呼叫，可以整合外部工具和 API。這些呼叫可以同時進行，也可以依序執行。
- *程式碼生成*——這個模型在 Python、Java、TypeScript 和 C++ 的生成上表現優異。


在這個範例中，我們使用 Mistral Large 2 來對一份文字文件執行 RAG 模式。問題是用韓文寫的，內容是詢問作者在大學之前的活動。

這裡用 Cohere Embeddings Model 來產生文字文件和問題的嵌入向量。這個範例中，向量儲存庫採用 faiss 這個 Python 套件。

傳送給 Mistral 模型的提示內容同時包含問題和與問題相似的檢索片段。模型接著會以自然語言給出回應。


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small 是 Mistral 系列模型中屬於高階／企業級別的另一款模型。顧名思義，這是一款小型語言模型（SLM）。使用 Mistral Small 的優點包括：
- 相較於 Mistral Large 和 NeMo 等 Mistral LLMs，能大幅節省成本——價格降低 80%
- 低延遲——回應速度比 Mistral 的 LLMs 更快
- 彈性高——可在不同環境中部署，對資源需求的限制較少

Mistral Small 非常適合用於：
- 文字相關任務，例如摘要、情感分析和翻譯
- 由於成本效益高，適合需要頻繁請求的應用
- 低延遲的程式碼任務，例如程式碼審查和建議


## 比較 Mistral Small 和 Mistral Large

為了展示 Mistral Small 和 Large 在延遲上的差異，請執行以下區塊。

你應該會看到回應時間大約有 3-5 秒的差距。同時也可以注意在相同提示下，回應的長度和風格會有所不同。


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

和本課程討論的另外兩個模型相比，Mistral NeMo 是唯一一個擁有 Apache2 授權且免費的模型。

它被視為 Mistral 早期開源 LLM——Mistral 7B 的升級版。

NeMo 模型的其他特色包括：

- *更高效的分詞方式：* 這個模型採用了 Tekken 分詞器，而不是更常見的 tiktoken。這讓它在多語言和程式碼處理上有更好的效能。

- *可微調：* 基礎模型開放給用戶進行微調。這讓需要微調的應用場景有更大的彈性。

- *原生函式呼叫* —— 和 Mistral Large 一樣，這個模型也經過函式呼叫的訓練。這讓它成為首批支援這項功能的開源模型之一。


## Mistral NeMo

和本課程討論的另外兩個模型相比，Mistral NeMo 是唯一一個擁有 Apache2 授權且免費的模型。

它被視為 Mistral 早期開源 LLM——Mistral 7B 的升級版。

NeMo 模型的其他特色包括：

- *更高效的分詞方式：* 這個模型採用了 Tekken 分詞器，而不是更常見的 tiktoken。這讓它在多語言和程式碼處理上有更好的表現。

- *可微調：* 基礎模型開放給用戶進行微調。這讓需要微調的應用場景有更多彈性。

- *原生函式呼叫* —— 和 Mistral Large 一樣，這個模型也經過函式呼叫的訓練。這讓它成為首批支援這項功能的開源模型之一。


### 比較分詞器

在這個範例中，我們會看看 Mistral NeMo 如何處理分詞，並與 Mistral Large 做比較。

兩個範例都使用相同的提示詞，但你會發現 NeMo 回傳的 token 數量比 Mistral Large 少。


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## 學習不止於此，繼續你的旅程

完成這堂課後，歡迎前往我們的 [生成式 AI 學習專區](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，持續提升你對生成式 AI 的了解！



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言之文件應視為具權威性的來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
